Вот описание двух финальных скриптов, которые ты используешь в пайплайне PhotoMaps:

🛠️ Скрипт 1: Сжатие изображений и генерация CSV (compress_and_index.py)

Задача:
Обрабатывает файлы из images/:
	•	Извлекает координаты и дату (EXIF + JSON)
	•	Если координаты есть — сохраняет сжатую копию в photos/
	•	Переименовывает файл по шаблону IMG_YYYYMMDD_hash.jpg, если дата не указана в имени
	•	Добавляет информацию (имя, координаты, дата) в csv/photos.csv
	•	Автоматически разбивает photos.csv на файлы по годам: photos_2020.csv, photos_2021.csv и т.д.
	•	Все отбраковки (без координат или ошибок) — в csv/errors_combined.csv

Вход:
	•	images/*.jpg|.jpeg
	•	(опционально) .json рядом с фото (.supplemental-metadata.json)

Выход:
	•	photos/*.jpg — сжатые изображения с гарантированными координатами
	•	csv/photos.csv — полный набор метаданных
	•	csv/photos_YYYY.csv — разбитые по годам
	•	csv/errors_combined.csv — лог ошибок

🎨 Скрипт 2: Генерация миниатюр (generate_thumbnails.py)

Задача:
Создает круглые миниатюры 64x64 PNG для каждого сжатого фото из photos/, окрашивает рамку по году.

Особенности:
	•	Превью оформлено как круг с цветной рамкой
	•	Цвет зависит от года (из палитры)
	•	Названия .jpg/.jpeg → .png

Вход:
	•	photos/*.jpg
	•	csv/photos.csv (для определения года)

Выход:
	•	thumbnails/*.png — иконки, отображаемые на карте

🔄 Обновлённый пайплайн
	1.	images/ — исходные JPG-файлы (и .json при наличии)
	2.	▶️ Скрипт 1: compress_and_index.py
	•	➡️ photos/ (JPEG, компрессия)
	•	➡️ csv/ (метаданные)
	3.	▶️ Скрипт 2: generate_thumbnails.py
	•	➡️ thumbnails/ (иконки PNG)
	4.	🗺️ Карта отображает точки с превью и попапами, данные из csv/photos_*.csv


In [ ]:
# -- script to study our images. Retrieves how many files of which types are there in source folder images/--

In [2]:
import os
import re
from collections import Counter

FOLDER = "images"  # замени на абсолютный путь при необходимости
files = os.listdir(FOLDER)

# === Расширения
ext_counts = Counter()
suffix_counts = Counter()
edited_count = 0
copy_count = 0

for f in files:
    lower = f.lower()
    if lower.endswith(('.jpg', '.jpeg', '.heic')):
        ext = os.path.splitext(f)[1].lower()
        ext_counts[ext] += 1

        # дубликаты: IMG_1234(1).JPG
        if re.search(r'\(\d+\)\.(jpe?g|heic)$', lower):
            suffix_counts['duplicate'] += 1

        # -edited.JPG
        if '-edited' in lower:
            edited_count += 1

        # " copy" в имени файла
        if re.search(r' copy(\s\d+)?\.(jpe?g|heic)$', lower):
            copy_count += 1

    elif lower.endswith('.json'):
        ext_counts['.json'] += 1

# === Вывод
print(f"📁 Анализ папки: {FOLDER}")
print("===================================")
for ext in ['.jpg', '.jpeg', '.heic', '.json']:
    print(f"{ext.upper():>6}: {ext_counts[ext]}")

print(f"🌀 Дубликатов с (1), (2)...: {suffix_counts['duplicate']}")
print(f"📋 Дубликатов copy         : {copy_count}")
print(f"✂️  -edited файлов         : {edited_count}")
print("===================================")
print(f"📦 Всего файлов: {len(files)}")

📁 Анализ папки: images
  .JPG: 2185
 .JPEG: 20
 .HEIC: 6
 .JSON: 2068
🌀 Дубликатов с (1), (2)...: 5
📋 Дубликатов copy         : 4
✂️  -edited файлов         : 146
📦 Всего файлов: 4282


In [ ]:
# -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234(1).JPG, IMG_1234(2).JPG and their json (if any)

In [4]:
import os
import re
import json

# === 📁 Папка с изображениями
folder = "images"

# === 🎯 Регулярка для поиска неправильных JSON
pattern = re.compile(r"(.*)\.supplemental-metadata\((\d+)\)\.json", re.IGNORECASE)

# === 🔄 Поиск всех JSON с неправильным именем
json_files = [f for f in os.listdir(folder) if pattern.match(f)]

# === 🔎 Логируем найденные файлы
if json_files:
    print(f"🔍 Найдены неправильные JSON файлы:")
    for f in json_files:
        print(f" - {f}")
else:
    print("⚠️ Не найдено файлов для переименования!")

for fname in json_files:
    json_path = os.path.join(folder, fname)
    
    # === ✅ Проверяем, что файл существует
    if not os.path.exists(json_path):
        print(f"❌ Ошибка: файл {json_path} не найден перед переименованием!")
        continue

    print(f"🔄 Обработка файла: {json_path}")
    
    # === 🔍 Получаем данные из регулярного выражения
    match = pattern.match(fname)
    base_name, index = match.groups()
    print(f"   → Базовое имя: {base_name}, Индекс: {index}")
    
    # === 📝 Чистим название от расширений, чтобы не дублировались
    if base_name.lower().endswith('.jpg'):
        base_name = base_name[:-4]
    elif base_name.lower().endswith('.jpeg'):
        base_name = base_name[:-5]

    # === 📝 Формируем правильное имя
    correct_name = f"{base_name}({index}).JPG.supplemental-metadata.json"
    correct_path = os.path.join(folder, correct_name)
    
    print(f"🔄 Переименование: {json_path} → {correct_path}")
    
    # === 🔄 Переименование
    try:
        os.rename(json_path, correct_path)
        
        # === ✅ Проверяем, что файл появился после переименования
        if not os.path.exists(correct_path):
            print(f"❌ Ошибка: файл {correct_path} не появился после переименования!")
            continue
        
    except Exception as e:
        print(f"❌ Ошибка при переименовании: {e}")
        continue
    
    # === 🔄 Читаем JSON и исправляем title
    try:
        with open(correct_path, "r", encoding="utf-8") as file:
            metadata = json.load(file)
        
        # === Новое значение для title
        old_title = metadata.get("title", "")
        new_title = f"{base_name}({index}).JPG"
        
        if old_title != new_title:
            print(f"🔄 Исправление title: {old_title} → {new_title}")
            metadata["title"] = new_title
            
            # === 📝 Записываем обратно
            with open(correct_path, "w", encoding="utf-8") as file:
                json.dump(metadata, file, ensure_ascii=False, indent=4)
        else:
            print(f"✅ Title уже корректный: {old_title}")
    
    except Exception as e:
        print(f"❌ Ошибка при обработке JSON: {correct_path} → {e}")

🔍 Найдены неправильные JSON файлы:
 - EFFECTS.jpg.supplemental-metadata(1).json
 - IMG_2822.JPG.supplemental-metadata(1).json
 - IMG_0128.JPG.supplemental-metadata(1).json
🔄 Обработка файла: images/EFFECTS.jpg.supplemental-metadata(1).json
   → Базовое имя: EFFECTS.jpg, Индекс: 1
🔄 Переименование: images/EFFECTS.jpg.supplemental-metadata(1).json → images/EFFECTS(1).JPG.supplemental-metadata.json
🔄 Исправление title: EFFECTS.jpg → EFFECTS(1).JPG
🔄 Обработка файла: images/IMG_2822.JPG.supplemental-metadata(1).json
   → Базовое имя: IMG_2822.JPG, Индекс: 1
🔄 Переименование: images/IMG_2822.JPG.supplemental-metadata(1).json → images/IMG_2822(1).JPG.supplemental-metadata.json
🔄 Исправление title: IMG_2822.JPG → IMG_2822(1).JPG
🔄 Обработка файла: images/IMG_0128.JPG.supplemental-metadata(1).json
   → Базовое имя: IMG_0128.JPG, Индекс: 1
🔄 Переименование: images/IMG_0128.JPG.supplemental-metadata(1).json → images/IMG_0128(1).JPG.supplemental-metadata.json
🔄 Исправление title: IMG_0128.JPG → I

In [ ]:
# -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234 copy.JPG, IMG_1234 copy copy.JPG and their json (if any)

In [6]:
import os
import re
import json
from PIL import Image, ExifTags

# === 📁 Папка с изображениями
folder = "images"

# === 🗂️ Проверяем, существует ли папка
if not os.path.exists(folder):
    print(f"❌ Папка '{folder}' не найдена. Создаю её...")
    os.makedirs(folder)
    print(f"✅ Папка '{folder}' создана. Помести в неё файлы для обработки.")
    exit()

# === 👁️ Проверка на наличие GPS в EXIF
def has_gps(exif):
    gps_tag_id = [k for k, v in ExifTags.TAGS.items() if v == "GPSInfo"]
    if not gps_tag_id:
        return False
    gps_data = exif.get(gps_tag_id[0])
    return bool(gps_data)

# === 🎯 Регулярное выражение для поиска цепочек " copy"
pattern = re.compile(r"^(.+?)( copy)+(\s\d+)?(\..+)$", re.IGNORECASE)
candidates = [f for f in os.listdir(folder) if pattern.search(f) and f.lower().endswith(('.jpg', '.jpeg', '.png', '.heic'))]

# === 🗂️ Списки результатов
not_found_json = []
updated_json_files = []
missing_gps_and_json = []
files_with_gps = []

print(f"🔎 Найдено файлов с меткой ' copy': {len(candidates)}")

for fname in candidates:
    base, ext = os.path.splitext(fname)
    original_base = base.replace(' copy', '')
    
    # === 🔄 Формируем имена JSON для всех регистров
    json_candidates = [
        f"{original_base}.jpg.supplemental-metadata copy.json",
        f"{original_base}.jpeg.supplemental-metadata copy.json",
        f"{original_base}.JPG.supplemental-metadata copy.json",
        f"{original_base}.JPEG.supplemental-metadata copy.json",
    ]
    
    # === 🔄 Ищем существующие JSON
    existing_jsons = [j for j in json_candidates if os.path.exists(os.path.join(folder, j))]
    
    if existing_jsons:
        json_path = os.path.join(folder, existing_jsons[0])
        print(f"\n✅ JSON найден для: {fname} → {existing_jsons[0]}")

        # === 📝 Переименовываем в правильный формат
        new_json_name = f"{fname}.supplemental-metadata.json"
        new_json_path = os.path.join(folder, new_json_name)
        os.rename(json_path, new_json_path)

        # === 🔄 Обновляем "title"
        with open(new_json_path, "r", encoding="utf-8") as file:
            metadata = json.load(file)

        original_title = metadata.get("title", "")
        if original_title != fname:
            print(f"🔄 Обновление 'title' в JSON: {original_title} → {fname}")
            metadata["title"] = fname
        
            # === 📝 Сохраняем обратно
            with open(new_json_path, "w", encoding="utf-8") as file:
                json.dump(metadata, file, ensure_ascii=False, indent=4)

        updated_json_files.append(fname)
        print(f"✅ JSON для {fname} успешно обработан → {new_json_path}")
    else:
        print(f"\n❌ JSON не найден для: {fname}")
        not_found_json.append(fname)
        continue

    # === ✅ 1. Если есть EXIF — пропускаем
    img_path = os.path.join(folder, fname)
    try:
        img = Image.open(img_path)
        exif = img._getexif() or {}
        if has_gps(exif):
            files_with_gps.append(fname)
            print(f"🌍 EXIF-координаты найдены в {fname}, JSON не требуется.")
            continue
    except Exception as e:
        missing_gps_and_json.append(f"{fname} (ошибка чтения EXIF: {e})")
        print(f"❌ Ошибка чтения EXIF: {fname} → {e}")
        continue

# === 📝 Итоги
print("\n=== 📋 РЕЗУЛЬТАТЫ ОБРАБОТКИ ===")
print(f"🔍 Найдено файлов-копий: {len(candidates)}")
print(f"📌 Файлов, содержащих EXIF: {len(files_with_gps)}")
print(f"✅ Успешно обработано JSON файлов: {len(updated_json_files)}")
print(f"❌ JSON не найден для битмапов: {len(not_found_json)}")

# === 📂 Полный список всех найденных файлов-копий
print("\n📂 Полный список всех найденных файлов-копий:")
for f in candidates:
    print(f" - {f}")

if not_found_json:
    print("\n⛔ Список не найденных JSON для изображений:")
    for f in not_found_json:
        print(f" - {f}")

🔎 Найдено файлов с меткой ' copy': 4

✅ JSON найден для: IMG_0275 copy.JPG → IMG_0275.jpg.supplemental-metadata copy.json
🔄 Обновление 'title' в JSON: IMG_0275.JPG → IMG_0275 copy.JPG
✅ JSON для IMG_0275 copy.JPG успешно обработан → images/IMG_0275 copy.JPG.supplemental-metadata.json

✅ JSON найден для: IMG_0302 copy.JPG → IMG_0302.jpg.supplemental-metadata copy.json
🔄 Обновление 'title' в JSON: IMG_0302.JPG → IMG_0302 copy.JPG
✅ JSON для IMG_0302 copy.JPG успешно обработан → images/IMG_0302 copy.JPG.supplemental-metadata.json

✅ JSON найден для: IMG_0860 copy.JPG → IMG_0860.jpg.supplemental-metadata copy.json
🔄 Обновление 'title' в JSON: IMG_0860.JPG → IMG_0860 copy.JPG
✅ JSON для IMG_0860 copy.JPG успешно обработан → images/IMG_0860 copy.JPG.supplemental-metadata.json

✅ JSON найден для: IMG_0795 copy.JPG → IMG_0795.jpg.supplemental-metadata copy.json
🔄 Обновление 'title' в JSON: IMG_0795.JPG → IMG_0795 copy.JPG
✅ JSON для IMG_0795 copy.JPG успешно обработан → images/IMG_0795 copy.JP

In [ ]:
# -- scripts to convert HEIC files into jpgs -- 

In [8]:
import os

# 📁 Укажи путь к папке с изображениями
folder = "images"

# 🔍 Собираем все .HEIC файлы
heic_files = [f for f in os.listdir(folder) if f.lower().endswith(".heic")]

invalid_files = []

for fname in heic_files:
    path = os.path.join(folder, fname)
    try:
        with open(path, "rb") as f:
            header = f.read(512)
            if b"ftyp" not in header:
                invalid_files.append(fname)
    except Exception as e:
        invalid_files.append(f"{fname} (ошибка чтения: {e})")

# 📋 Выводим список подозрительных файлов
print(f"\n🔍 Проверка .HEIC в папке {folder}")
print(f"Найдено: {len(heic_files)} HEIC файлов")
print(f"❌ Без 'ftyp': {len(invalid_files)}")
for f in invalid_files:
    print(f" - {f}")


🔍 Проверка .HEIC в папке images
Найдено: 6 HEIC файлов
❌ Без 'ftyp': 6
 - IMG_1894.HEIC
 - IMG_1899.HEIC
 - IMG_1811.HEIC
 - IMG_2005.HEIC
 - IMG_1890.HEIC
 - IMG_1901.HEIC


In [10]:
import os
import subprocess
import json

# === 📁 Папка с изображениями
folder = "images"

# === 🔍 Находим все HEIC файлы
heic_files = [f for f in os.listdir(folder) if f.lower().endswith(".heic")]
converted = []
failed = []

# === 🔄 Конвертация HEIC → JPG и переименование JSON
for fname in heic_files:
    in_path = os.path.join(folder, fname)
    out_name = os.path.splitext(fname)[0] + ".jpg"
    out_path = os.path.join(folder, out_name)

    try:
        result = subprocess.run(
            ["sips", "-s", "format", "jpeg", in_path, "--out", out_path],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            # 🗑️ Удаляем исходный HEIC
            os.remove(in_path)
            converted.append(out_name)

            # === 🔄 Переименование JSON
            json_name = f"{os.path.splitext(fname)[0]}.HEIC.supplemental-metadata.json"
            new_json_name = f"{os.path.splitext(fname)[0]}.JPG.supplemental-metadata.json"

            json_path = os.path.join(folder, json_name)
            new_json_path = os.path.join(folder, new_json_name)

            if os.path.exists(json_path):
                # === 🔄 Переименование JSON
                os.rename(json_path, new_json_path)
                print(f"🔄 Переименован JSON: {json_name} → {new_json_name}")

                # === 📝 Исправляем "title" внутри JSON
                try:
                    with open(new_json_path, "r", encoding="utf-8") as file:
                        metadata = json.load(file)

                    # Обновляем поле "title"
                    metadata["title"] = out_name

                    # Сохраняем обратно
                    with open(new_json_path, "w", encoding="utf-8") as file:
                        json.dump(metadata, file, ensure_ascii=False, indent=4)

                    print(f"✅ Обновлен 'title' в JSON: {out_name}")
                except Exception as e:
                    print(f"❌ Ошибка при обновлении JSON {new_json_path}: {e}")
            else:
                print(f"⚠️ JSON не найден: {json_name}")
        else:
            failed.append((fname, result.stderr.strip()))
    except Exception as e:
        failed.append((fname, str(e)))

# === 📝 Результат
print(f"\n✅ Конвертировано и удалено: {len(converted)} HEIC → JPG")
if failed:
    print(f"❌ Ошибки: {len(failed)}")
    for fname, msg in failed:
        print(f" - {fname}: {msg}")

🔄 Переименован JSON: IMG_1894.HEIC.supplemental-metadata.json → IMG_1894.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_1894.jpg
🔄 Переименован JSON: IMG_1899.HEIC.supplemental-metadata.json → IMG_1899.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_1899.jpg
🔄 Переименован JSON: IMG_1811.HEIC.supplemental-metadata.json → IMG_1811.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_1811.jpg
🔄 Переименован JSON: IMG_2005.HEIC.supplemental-metadata.json → IMG_2005.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_2005.jpg
🔄 Переименован JSON: IMG_1890.HEIC.supplemental-metadata.json → IMG_1890.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_1890.jpg
🔄 Переименован JSON: IMG_1901.HEIC.supplemental-metadata.json → IMG_1901.JPG.supplemental-metadata.json
✅ Обновлен 'title' в JSON: IMG_1901.jpg

✅ Конвертировано и удалено: 6 HEIC → JPG


In [ ]:
# --- images>photos converter  - MAIN script --

In [12]:
import os
from PIL import Image, ExifTags
import pillow_heif
import json
import re
import pandas as pd
from tqdm import tqdm
import glob
import math

# === 📁 Папки
ROOT = os.path.abspath(".")
SRC_FOLDER = os.path.join(ROOT, "images")
OUT_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
LOG_FILE = os.path.join(CSV_FOLDER, "import_log.txt")
os.makedirs(OUT_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)

CSV_MAIN = os.path.join(CSV_FOLDER, "photos.csv")
CSV_MISSING = os.path.join(CSV_FOLDER, "missing_coords.csv")

# === EXIF utils
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m) / 60 + float(s) / 3600

def extract_gps(exif):
    gps_raw = {}
    for tag_id, val in exif.items():
        tag = ExifTags.TAGS.get(tag_id)
        if tag == "GPSInfo":
            for key in val:
                gps_tag = ExifTags.GPSTAGS.get(key)
                gps_raw[gps_tag] = val[key]
    if 'GPSLatitude' in gps_raw and 'GPSLongitude' in gps_raw:
        try:
            lat = convert_to_degrees(gps_raw['GPSLatitude'])
            if gps_raw.get('GPSLatitudeRef', 'N') != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_raw['GPSLongitude'])
            if gps_raw.get('GPSLongitudeRef', 'E') != 'E':
                lon = -lon
            return lat, lon
        except:
            return None, None
    return None, None

def extract_date(exif):
    date_str = exif.get(36867) or exif.get(306)
    if date_str:
        match = re.match(r"(\d{4}):(\d{2}):(\d{2})", date_str)
        if match:
            return match.group(1), match.group(2), match.group(3)
    return None, None, None

Image.init()

def spiral_coords(lat, lon, index, step=0.0002):
    angle = index * (math.pi / 3)
    radius = step * (1 + index // 6)
    return lat + radius * math.cos(angle), lon + radius * math.sin(angle)

image_files = [f for f in os.listdir(SRC_FOLDER)
               if f.lower().endswith(('.jpg', '.jpeg', '.heic')) and "-edited" not in f.lower()]

main_records = []
missing_records = []
log_entries = []

# === Прогресс-бар
for fname in tqdm(image_files, desc="📦 Обработка изображений", ncols=80):
    in_path = os.path.join(SRC_FOLDER, fname)
    ext = os.path.splitext(fname)[1].lower()

    try:
        if ext == ".heic":
            heif_file = pillow_heif.read_heif(in_path)
            image = Image.frombytes(
                heif_file.mode, heif_file.size, heif_file.data, "raw")
            exif = image.getexif()
            log_entries.append(f"[HEIC] Converted and loaded: {fname}")
        else:
            image = Image.open(in_path)
            exif = image._getexif() or {}
            log_entries.append(f"[IMG] Loaded: {fname}")

        lat, lon = extract_gps(exif)
        year, month, day = extract_date(exif)
        source_type = "EXIF"

        # === 🔄 Поиск JSON для всех вариантов
        json_candidates = glob.glob(in_path + ".supplemental*.json")
        
        if not json_candidates:
            alt_path = in_path.replace(".jpg", ".JPG")
            json_candidates = glob.glob(alt_path + ".supplemental*.json")
        
        if not json_candidates:
            alt_path = in_path.replace(".jpeg", ".JPEG")
            json_candidates = glob.glob(alt_path + ".supplemental*.json")
        
        if json_candidates:
            try:
                with open(json_candidates[0], "r", encoding="utf-8") as jf:
                    meta = json.load(jf)
                if (lat is None or lon is None) and "geoData" in meta:
                    lat = meta["geoData"].get("latitude")
                    lon = meta["geoData"].get("longitude")
                    if lat and lon:
                        source_type = "JSON"
                        log_entries.append(f"[JSON] Used metadata for: {fname}")
            except Exception as e:
                log_entries.append(f"[ERROR] JSON read failed for {fname}: {e}")
                pass

        if not (year and month and day):
            year, month, day = "2099", "01", "01"

        if not re.search(r'IMG_\d{8}_', fname):
            base_name = f"IMG_{year}{month}{day}_{abs(hash(fname)) % 10**6}.jpg"
        else:
            base_name = fname.replace(".heic", ".jpg").replace(".HEIC", ".jpg")

        out_path = os.path.join(OUT_FOLDER, base_name)
        image.convert("RGB").save(out_path, "JPEG", quality=85)
        log_entries.append(f"[SAVE] {fname} → {base_name}")

        row = {
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': lat,
            'longitude': lon,
            'year': year,
            'month': month,
            'day': day,
            'source_path': fname,
            'source_type': source_type
        }

        if lat and lon:
            main_records.append(row)
        else:
            missing_records.append(row)
            log_entries.append(f"[MISSING] {fname} → No coordinates")

    except Exception as e:
        missing_records.append({
            'filename': fname,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': None,
            'longitude': None,
            'year': None,
            'month': None,
            'day': None,
            'error': str(e),
            'source_path': fname,
            'source_type': None
        })
        log_entries.append(f"[ERROR] {fname}: {e}")

# === Сохраняем в CSV
pd.DataFrame(main_records).to_csv(CSV_MAIN, index=False)
pd.DataFrame(missing_records).to_csv(CSV_MISSING, index=False)

print(f"\n✅ С координатами: {len(main_records)} → {CSV_MAIN}")
print(f"⚠️ Без координат : {len(missing_records)} → {CSV_MISSING}")
print(f"📁 Фото сохранены в: {OUT_FOLDER}")
print(f"📝 Лог записан в: {LOG_FILE}")
print("🌍 Джиттер применён для повторяющихся координат")

print("\n✅ Завершено. Результаты в photos.csv и missing_coords.csv")


📦 Обработка изображений: 100%|█████████████| 2065/2065 [06:49<00:00,  5.04it/s]



✅ С координатами: 2065 → /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv
⚠️ Без координат : 0 → /Users/mloktionov/PycharmProjects/PhotoMaps/csv/missing_coords.csv
📁 Фото сохранены в: /Users/mloktionov/PycharmProjects/PhotoMaps/photos
📝 Лог записан в: /Users/mloktionov/PycharmProjects/PhotoMaps/csv/import_log.txt
🌍 Джиттер применён для повторяющихся координат

✅ Завершено. Результаты в photos.csv и missing_coords.csv


In [ ]:
# ------ Generating preview thumbnails in circles of a certain color

In [15]:
import os
from PIL import Image, ImageDraw
from tqdm import tqdm

# === 📁 Пути ===
ROOT = os.path.abspath(".")
PHOTO_FOLDER = os.path.join(ROOT, "photos")
THUMBNAIL_FOLDER = os.path.join(ROOT, "thumbnails")
os.makedirs(THUMBNAIL_FOLDER, exist_ok=True)

# === 🖼️ Параметры
SIZE = (64, 64)
BORDER_WIDTH = 4
BORDER_COLOR = "#888888"

# === 🔄 Обработка
jpg_files = [f for f in os.listdir(PHOTO_FOLDER) if f.lower().endswith((".jpg", ".jpeg"))]
count = 0
skipped = 0

for fname in tqdm(jpg_files, desc="🎨 Превью", ncols=80):
    try:
        in_path = os.path.join(PHOTO_FOLDER, fname)
        out_name = fname.lower().replace(".jpg", ".png").replace(".jpeg", ".png")
        out_path = os.path.join(THUMBNAIL_FOLDER, out_name)

        img = Image.open(in_path).convert("RGBA")
        img = img.resize(SIZE, Image.LANCZOS)

        mask = Image.new("L", SIZE, 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((0, 0, SIZE[0], SIZE[1]), fill=255)

        result = Image.new("RGBA", SIZE)
        result.paste(img, (0, 0), mask)

        draw = ImageDraw.Draw(result)
        inset = BORDER_WIDTH // 2
        draw.ellipse(
            (inset, inset, SIZE[0] - inset - 1, SIZE[1] - inset - 1),
            outline=BORDER_COLOR, width=BORDER_WIDTH
        )

        result.save(out_path, "PNG")
        count += 1

    except Exception as e:
        skipped += 1
        print(f"❌ {fname}: {e}")

print(f"\n✅ Превью создано: {count}")
print(f"⚠️ Пропущено: {skipped}")
print(f"📁 Сохранено в: {THUMBNAIL_FOLDER}")

🎨 Превью: 100%|████████████████████████████| 2065/2065 [04:34<00:00,  7.53it/s]


✅ Превью создано: 2065
⚠️ Пропущено: 0
📁 Сохранено в: /Users/mloktionov/PycharmProjects/PhotoMaps/thumbnails


In [ ]:
----